In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [ ]:
x=pd.read_csv('PATH/PPMI_Dataset/Subject_Characteristics/Screening___Demographics.csv')
x[x['APPRDX']==1]

In [ ]:
x=pd.read_csv('PATH/PPMI_Dataset/Subject_Characteristics/PPMI_Baseline_Data_02Jul2018.csv')
PD_patlist=x[x['APPRDX']==1]['PATNO']

In [ ]:
months=['BL','V04','V06','V08','V10']
files=[]
for period in months:
    
    y=pd.read_csv('PATH/PPMI_Dataset/Motor_Assessments/MDS_UPDRS_Part_III.csv')
    y=y[y['PATNO'].isin(PD_patlist)]
    y=y[y['EVENT_ID']==period]
    y=y[(y['PAG_NAME']=='NUPDRS3')]
    
    y=y.iloc[:,np.r_[2,11:44]]
    y=y.sort_values(by=['PATNO']).reset_index()
    files.append(y.drop(columns='index'))


In [ ]:
for df in files:
    df['Total_UPDRS3']=df.loc[:,'NP3SPCH':'NP3RTCON'].sum(axis=1,skipna=False)
    

In [ ]:
prog=pd.DataFrame()
prog['PATNO']=files[0]['PATNO']
prog['BL_UPDRS3']=files[0]['Total_UPDRS3']


In [ ]:
prog=prog.merge(files[1].loc[:,['PATNO','Total_UPDRS3']],how='left',on='PATNO')
prog=prog.rename(columns={'Total_UPDRS3':'year1_UPDRS3'})

prog=prog.merge(files[2].loc[:,['PATNO','Total_UPDRS3']],how='left',on='PATNO')
prog=prog.rename(columns={'Total_UPDRS3':'year2_UPDRS3'})

prog=prog.merge(files[3].loc[:,['PATNO','Total_UPDRS3']],how='left',on='PATNO')
prog=prog.rename(columns={'Total_UPDRS3':'year3_UPDRS3'})

prog=prog.merge(files[4].loc[:,['PATNO','Total_UPDRS3']],how='left',on='PATNO')
prog=prog.rename(columns={'Total_UPDRS3':'year4_UPDRS3'})


In [ ]:
prog['Diff_BL_1yr']=prog['year1_UPDRS3']-prog['BL_UPDRS3']
prog['Diff_BL_2yr']=prog['year2_UPDRS3']-prog['BL_UPDRS3']
prog['Diff_BL_3yr']=prog['year3_UPDRS3']-prog['BL_UPDRS3']
prog['Diff_BL_4yr']=prog['year4_UPDRS3']-prog['BL_UPDRS3']


In [ ]:
prog['Diff_BL_4yr'].dropna()

In [ ]:
yr='4'
sns.distplot(prog['Diff_BL_'+yr+'yr'],kde_kws={'shade':True,'linewidth':2})
plt.savefig('Diff_BL_'+yr+'yr')


x=prog.loc[:,['PATNO','Diff_BL_'+yr+'yr']].dropna().sort_values(by=['Diff_BL_'+yr+'yr'],ascending=False)
group_early=x[x['Diff_BL_'+yr+'yr']>20]['PATNO']
group_others=x[x['Diff_BL_'+yr+'yr']<15]['PATNO']
print('20% are early progressor. Patients available: ',len(group_early))
print('80% are other patient. Patients available: ',len(group_others))

plt.hist(prog['Diff_BL_'+yr+'yr'],bins=np.arange(-40,81,5),color='b')


In [ ]:
BL_Feat=pd.DataFrame()
BL_Feat['PATNO']=PD_patlist

## Vital signs


In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Medical_History/Vital_Signs.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x=x.iloc[:,np.r_[2,6:9,10:16]]
x['BMI']=x['WGTKG']/((x['HTCM']/100)*(x['HTCM']/100))
BL_Feat=BL_Feat.merge(x.loc[:,['PATNO','BMI','SYSSUP','HRSUP']],on='PATNO',how='left')

In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Subject_Characteristics/PPMI_Baseline_Data_02Jul2018.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x=x.iloc[:,np.r_[1,4,6,7,14,15,16]]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')
BL_Feat

# Motor Symptoms
## MDS-UPDRS I
### aggregate scores: Cognitive, sleep, Autonomic nervous system and total updrs 1

In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Motor_Assessments/MDS_UPDRS_Part_I.csv')
y=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Motor_Assessments/MDS_UPDRS_Part_I__Patient_Questionnaire.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']

y=y[y['PATNO'].isin(PD_patlist)]
y=y[y['EVENT_ID']=='BL']
x=x.iloc[:,np.r_[2,7:13]]
y=y.iloc[:,np.r_[2,7:14]]
x=x.merge(y,how='outer',on='PATNO')
x['Cognitive']=x.loc[:,['NP1COG','NP1HALL','NP1DPRS','NP1ANXS','NP1APAT','NP1DDS']].sum(axis=1,skipna=False)
x['Sleep']=x.loc[:,['NP1SLPN','NP1SLPD']].sum(axis=1,skipna=False)
x['Autonomic_Nervous_System']=x.loc[:,['NP1URIN','NP1CNST','NP1LTHD','NP1FATG']].sum(axis=1,skipna=False)
x['Total_UPDRS1']=x.loc[:,'NP1COG':'NP1FATG'].sum(axis=1,skipna=False)

BL_Feat=BL_Feat.merge(x.iloc[:,np.r_[0,14:18]],on='PATNO',how='left')




## MDS-UPDRS II
### aggregate scores: Bulbar, common daily activity, bed, gait and total updrs 2

In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Motor_Assessments/MDS_UPDRS_Part_II__Patient_Questionnaire.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x['Bulbar']=x.loc[:,['NP2SPCH','NP2SALV','NP2SWAL','NP2EAT']].sum(axis=1,skipna=False)
x['Common_daily_act']= x.loc[:,['NP2DRES','NP2HYGN','NP2HWRT','NP2HOBB']].sum(axis=1,skipna=False)
x['Bed']=x.loc[:,['NP2TURN','NP2RISE']].sum(axis=1,skipna=False)
x['Gait']=x.loc[:,['NP2WALK','NP2FREZ']].sum(axis=1,skipna=False)
x['Total_UPDRS2']=x.loc[:,'NP2SPCH':'NP2FREZ'].sum(axis=1,skipna=False)
x=x.loc[:,['PATNO','Bulbar','Common_daily_act','Bed','Gait','Total_UPDRS2']]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')


## MDS-UPDRS III
### aggregate scores: Axial subscore 1, axial subscore 2, limb rigidity, limb bradykinesia, tremor, rest tremor, appendicular, left motor score, right motor score, asymmetry, total updrs 3 and total updrs

In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Motor_Assessments/MDS_UPDRS_Part_III.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x=x.iloc[:,np.r_[2,11:44]]
x['Axial_Sub_1']=x.loc[:,['NP3SPCH','NP3FACXP','NP3RIGN',
                     'NP3RISNG','NP3GAIT','NP3FRZGT',
                     'NP3PSTBL','NP3POSTR','NP3BRADY','NP3RTALJ']].sum(axis=1,skipna=False)
x['Axial_Sub_2']=x.loc[:,['NP3RISNG','NP3GAIT','NP3FRZGT',
                     'NP3PSTBL','NP3POSTR','NP3BRADY']].sum(axis=1,skipna=False)
x['Limb_Rig_Sub']=x.loc[:,['NP3RIGRU','NP3RIGLU','PN3RIGRL','NP3RIGLL']].sum(axis=1,skipna=False)
x['Limb_Brady_Sub']=x.loc[:,['NP3FTAPR','NP3FTAPL','NP3HMOVR','NP3HMOVL',
                             'NP3PRSPR','NP3PRSPL','NP3TTAPR','NP3TTAPL',
                            'NP3LGAGR','NP3LGAGL']].sum(axis=1,skipna=False)
x['Tremor_Sub']=x.loc[:,['NP3PTRMR','NP3PTRML','NP3KTRMR','NP3KTRML','NP3RTARU',
                          'NP3RTALU','NP3RTARL','NP3RTALL','NP3RTCON']].sum(axis=1,skipna=False)
x['Rest_Tremor_Sub']=x.loc[:,['NP3RTARU','NP3RTALU','NP3RTARL','NP3RTALL','NP3RTCON']].sum(axis=1,skipna=False)
x['Append_Sub']=x.loc[:,['Limb_Rig_Sub','Limb_Brady_Sub','Tremor_Sub']].sum(axis=1,skipna=False)
x['Left_Motor_Score']=x.loc[:,['NP3RIGLU','NP3RIGLL','NP3FTAPL','NP3HMOVL',
                                 'NP3PRSPL','NP3TTAPL','NP3LGAGL','NP3PTRML',
                                 'NP3KTRML','NP3RTALU','NP3RTALL']].sum(axis=1,skipna=False)
x['Right_Motor_Score']=x.loc[:,['NP3RIGRU','PN3RIGRL','NP3FTAPR','NP3HMOVR',
                                  'NP3PRSPR','NP3TTAPR','NP3LGAGR','NP3PTRMR',
                                  'NP3KTRMR','NP3RTARU','NP3RTARL']].sum(axis=1,skipna=False)
x['Diff']=x['Left_Motor_Score']-x['Right_Motor_Score']
x['Asymmetry']=np.abs(x['Diff'])

x['Total_UPDRS3']=x.loc[:,'NP3SPCH':'NP3RTCON'].sum(axis=1,skipna=False)

x=x.loc[:,['PATNO','Axial_Sub_1','Axial_Sub_2','Limb_Rig_Sub','Limb_Brady_Sub','Tremor_Sub','Rest_Tremor_Sub',
          'Append_Sub','Left_Motor_Score','Right_Motor_Score','Diff','Asymmetry','Total_UPDRS3']]

BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')
BL_Feat['Total_UPDRS']=BL_Feat['Total_UPDRS1']+BL_Feat['Total_UPDRS2']+BL_Feat['Total_UPDRS3']


## Modified Schwab and England ADL
### in the file is present the final total score, I used it.

In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Motor_Assessments/Modified_Schwab_+_England_ADL.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x=x.loc[:,['PATNO','MSEADLG']]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')


## PASE Household activity


In [ ]:
#NO DATA AT BL OR SCREENING
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Motor_Assessments/PASE_-_Household_Activity.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x

# Non- Motor Symptoms
## Benton Judgment of line orientation


In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Non-Motor-Asses/Neurophysiological/Benton_Judgment_of_Line_Orientation.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x=x.loc[:,['PATNO','DVS_JLO_MSSAE']]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')

## Cognitive categorization
### at BL there are data for 106 patients instead of 423, 75% of data missing. useless file.

In [ ]:
#106 on 423 = 75% of data missing on cognitive categorization
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Non-Motor-Asses/Cognition/Cognitive_Categorization.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x

## Epworth Sleepiness Scale
### In the original paper they suggest to made the sum of the 8 questions. I have used the sum.

In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Non-Motor-Asses/Sleep_Disorder/Epworth_Sleepiness_Scale.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x['Epworth_SUM']=x.loc[:,'ESS1':'ESS8'].sum(axis=1,skipna=False)
x=x.loc[:,['PATNO','Epworth_SUM']]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')


## Geriatric depression scale short
### In the original paper they suggest the sum of the question outcome. I have used it.

In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Non-Motor-Asses/NeuroBehavioral/Geriatric_Depression_Scale__Short_.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x=x.iloc[:,np.r_[2,6:21,]]
no=['GDSSATIS', 'GDSGSPIR', 'GDSHAPPY', 'GDSALIVE', 'GDSENRGY']
x['GDS_SUM']=np.zeros(len(x))
for i in no:
    x[i]=1-x[i]
x['GDS_SUM']=x.loc[:,'GDSSATIS':'GDSBETER'].sum(axis=1,skipna=False)
x=x.loc[:,['PATNO','GDS_SUM']]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')


## Hopkins verbal learning test
### Derived-Total Recall T-Score, Derived-Delayed Recall T-Score, Derived-Retention T-Score, Derived-Recog. Discrim. Index T-Score

In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Non-Motor-Asses/Neurophysiological/Hopkins_Verbal_Learning_Test.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x=x.iloc[:,np.r_[2,16]]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')
BL_Feat

## Letter number sequencing PD
### In the file there is the total score

In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Non-Motor-Asses/Neurophysiological/Letter_-_Number_Sequencing__PD_.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x=x.loc[:,['PATNO','DVS_LNS']]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')

## Montreal cognitive assessment


In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Non-Motor-Asses/Neurophysiological/Montreal_Cognitive_Assessment__MoCA_.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='SC']
x=x.loc[:,['PATNO','MCATOT']]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')

## Questionnaire for impulsive compulsive disorders

In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Non-Motor-Asses/NeuroBehavioral/QUIP_Current_Short.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x=x.iloc[:,np.r_[2,7:20]]
x['QUIP_Sum']=x.loc[:,'TMGAMBLE':'CNTRLDSM'].sum(axis=1,skipna=False)
x['QUIP_Buying']=x.loc[:,['TMBUY','CNTRLBUY']].sum(axis=1,skipna=False)
x['QUIP_Eating']=x.loc[:,['TMEAT','CNTRLEAT']].sum(axis=1,skipna=False)
x['QUIP_Gamble']=x.loc[:,['TMGAMBLE','CNTRLGMB']].sum(axis=1,skipna=False)
x['QUIP_Hobbies']=x.loc[:,['TMTORACT']].sum(axis=1,skipna=False)
x['QUIP_Punding']=x.loc[:,['TMTMTACT']].sum(axis=1,skipna=False)
x['QUIP_Sex']=x.loc[:,['TMSEX','CNTRLSEX']].sum(axis=1,skipna=False)
x['QUIP_Driving']=x.loc[:,['TMTRWD']].sum(axis=1,skipna=False)
                          
x=x.loc[:,['PATNO','QUIP_Sum']]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')


## REM Sleep disorder questionnaire


In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Non-Motor-Asses/Sleep_Disorder/REM_Sleep_Disorder_Questionnaire.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x=x.iloc[:,np.r_[2,7:28]]
x['REM_Sum']=x.loc[:,'DRMVIVID':'CNSOTH'].sum(axis=1,skipna=False)
x=x.loc[:,['PATNO','REM_Sum']]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')


## SCOPA-AUT: Scale for outcomes in PD for autonomic symptoms


In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Non-Motor-Asses/Autonomic/SCOPA-AUT.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
for i in x.loc[:,'SCAU1':'SCAU21'].columns:
    x[i]=x[i].replace(9,3)
for i in x.loc[:,'SCAU22':'SCAU25'].columns: 
    x[i]=x[i].replace(9,0)
x['SCOPA_TOT']=x.loc[:,'SCAU1':'SCAU25'].sum(axis=1)
x['SCOPA_Gastro']=x.loc[:,'SCAU1':'SCAU7'].sum(axis=1)
x['SCOPA_Urinary']=x.loc[:,'SCAU8':'SCAU13'].sum(axis=1)
x['SCOPA_Cardio']=x.loc[:,'SCAU14':'SCAU16'].sum(axis=1)
x['SCOPA_Pupillomotor']=x.loc[:,'SCAU19']
x['SCOPA_Thermoreg']=x.loc[:,['SCAU17','SCAU18','SCAU20','SCAU21']].sum(axis=1)
x['SCOPA_Sex']=x.loc[:,'SCAU22':'SCAU25'].sum(axis=1)
x=x.loc[:,['PATNO','SCOPA_TOT']]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')

## Lexical fluency


In [ ]:
#SIETE FAKE NON CI SONO i DATI ALLA BASELINE
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Non-Motor-Asses/Neurophysiological/Lexical_Fluency.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='SC']
x

## STAI, Stait trait anxiety inventory

In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Non-Motor-Asses/NeuroBehavioral/State-Trait_Anxiety_Inventory.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x=x.iloc[:,np.r_[2,6:46]]
rev=[1, 2, 5, 8, 10, 11, 15, 16, 19, 20, 21, 23, 26, 27, 30, 33, 34, 36, 39]
for (j,i) in zip(x.iloc[:,1:].columns,range(1,41)):
    if i in rev:
        x[j]=5-x[j]
x['STAI_Sum_S']=x.loc[:,'STAIAD1':'STAIAD20'].sum(axis=1,skipna=False)
x['STAI_Sum_T']=x.loc[:,'STAIAD21':'STAIAD40'].sum(axis=1,skipna=False)
x['STAI_Sum']=x['STAI_Sum_S']+x['STAI_Sum_T']
x=x.loc[:,['PATNO','STAI_Sum']]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')


## Symbol digit Modalities

In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Non-Motor-Asses/Neurophysiological/Symbol_Digit_Modalities.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x=x.loc[:,['PATNO','DVT_SDM']]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')


## University of Pennysylvania smell id test

In [ ]:
x=pd.read_csv('Desktop/Thesis/PPMI_Dataset/Non-Motor-Asses/Olfactory/University_of_Pennsylvania_Smell_ID_Test.csv')
x=x[x['PATNO'].isin(PD_patlist)]
x=x[x['EVENT_ID']=='BL']
x=x.loc[:,['PATNO','UPSITBK1','UPSITBK2','UPSITBK3','UPSITBK4']]
x['UPSIT_Sum']=x.loc[:,'UPSITBK1':'UPSITBK4'].sum(axis=1,skipna=False)
x=x.loc[:,['PATNO','UPSIT_Sum']]
BL_Feat=BL_Feat.merge(x,on='PATNO',how='left')


In [ ]:
var=['ageonset','agediag','DOMSIDE']
for i in var:
    BL_Feat[i]=BL_Feat[i].astype('float64')
print(BL_Feat.columns)
BL_Feat.info()

In [ ]:
BL_Feat=BL_Feat.merge(prog.loc[:,['PATNO','Diff_BL_4yr']].dropna(),how='left',on='PATNO')

In [ ]:
BL_Feat=BL_Feat[np.isnan(BL_Feat['Diff_BL_4yr'])==False]
BL_Feat

In [ ]:
BL_Feat.info()

In [ ]:
BL_Feat=BL_Feat.drop(columns=['Diff','Left_Motor_Score','Right_Motor_Score','Asymmetry',
                              'Total_UPDRS1','Total_UPDRS2','Total_UPDRS3','Total_UPDRS'])


In [ ]:
BL_Feat.info()

In [ ]:
early=BL_Feat[BL_Feat['PATNO'].isin(group_early)].copy()
others=BL_Feat[BL_Feat['PATNO'].isin(group_others)].copy()
early['Class']=1
others['Class']=0
whole=pd.concat([early,others])
whole=whole.sort_values(by=['PATNO']).reset_index().drop(columns=['index','Diff_BL_4yr'])
whole
#whole.to_csv('BL_features_1.csv')

In [ ]:
whole.describe()

# Outliers
## No medical reason to exclude few of them

In [ ]:
from sklearn.preprocessing import scale

X=whole.iloc[:,1:-1]
Y=whole.iloc[:,-1]
import seaborn as sns
var=scale(X)
warnings.filterwarnings('ignore')
for j in range(np.shape(var)[1]):
    plt.figure()
    plt.title(X.iloc[:,j].name)
    plt.boxplot(var[:,j][~np.isnan(var[:,j])])

In [ ]:
X.info()

# Drop row with missing values, Standardization, Division in train and test

In [ ]:
from sklearn.model_selection import train_test_split
#Drop row with missing values
ind=X[X.isna().any(axis=1)].index
X=X.drop(index=ind).reset_index().drop(columns='index')
Y=Y.drop(index=ind).reset_index().drop(columns='index')


    
#Standardization

for col in X.columns:
    if col=='gen' or col=='DOMSIDE':
        continue
    else:
        X[col]=(X[col]-X[col].mean())/X[col].std()
        
#Correlation analysis
corr=X.corr().abs()
sns.heatmap(X.corr())
feat=list(X.columns)
drop=[]
for i in range(len(corr)):
    for j in range(i+1,len(corr)):
            if np.abs(corr.iloc[i,j]) >0.8:
                print(feat[i],feat[j],corr.iloc[i,j])
                drop.append(feat[i])
                drop.append(feat[j])
drop=list(dict.fromkeys(drop))
keep=['age','Axial_Sub_1','Append_Sub','Tremor_Sub']
drop=list(set(drop)-set(keep))
X=X.drop(columns=drop)




In [ ]:
X.info()

In [ ]:
X[X['gen']==2]

In [ ]:
X['DVT_SDM'].unique()

In [ ]:
#Division in train and test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1,stratify=Y)

In [ ]:
print('X_train:',len(X_train))
print('X_test:',len(X_test))
print('y_train early:',len(y_train[y_train['Class']==1]))
print('y_train others:',len(y_train[y_train['Class']==0]))
print('y_test early:',len(y_test[y_test['Class']==1]))
print('y_test others:',len(y_test[y_test['Class']==0]))

# Models without feature selection

In [ ]:
#Model without feature selection
from sklearn.metrics import f1_score,confusion_matrix,precision_score,balanced_accuracy_score,recall_score,accuracy_score,roc_auc_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.dummy import DummyClassifier

models=[RandomForestClassifier(n_estimators=5,n_jobs=-1,class_weight='balanced_subsample',bootstrap=True,random_state=0),
       GaussianNB(),LogisticRegression(class_weight='balanced'),DummyClassifier(strategy='most_frequent'),
        SGDClassifier('log',class_weight='balanced',learning_rate='adaptive',eta0=0.001), DecisionTreeClassifier(class_weight='balanced',),
        AdaBoostClassifier(n_estimators=30,random_state=0),GradientBoostingClassifier(n_estimators=10,random_state=0)]
model_names=['RF','GNB','LOG_Reg','Dummy','SGDClass','DecTree','ADA','GBC']

for model,model_name in zip(models,model_names):
    print(model_name)
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    print('F1:',f1_score(y_test,y_pred))
    print('Precision:',precision_score(y_test,y_pred))
    print('Recall:',recall_score(y_test,y_pred))
    print('Confusion matrix:\n',confusion_matrix(y_test,y_pred))
    print('Accuracy:',balanced_accuracy_score(y_test,y_pred))
    try:
        print('Roc AUC:',roc_auc_score(y_test,y_pred))
    except:
        print('ROC AUC: Nan')
    print('\n')

# SMOTE on original training set

In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTENC,BorderlineSMOTE,ADASYN
from sklearn.metrics import roc_auc_score

models=[RandomForestClassifier(n_estimators=10,n_jobs=-1,bootstrap=True),
       GaussianNB(),LogisticRegression(),
        SGDClassifier('log',learning_rate='adaptive',eta0=0.001), DecisionTreeClassifier(),
        AdaBoostClassifier()]
model_names=['RF','GNB','LOG_Reg','SGDClass','DecTree','ADA']
for model,model_name in zip(models,model_names):
    print(model_name)
    print(Counter(y_train['Class']))
    oversample = SMOTENC(categorical_features=[1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,
                                              22,23,24,25,26,27,28,29,30,31],k_neighbors=1)
    X_train_s, y_train_s = oversample.fit_sample(X_train, y_train)
    print(Counter(y_train_s['Class']))
    model.fit(X_train_s,y_train_s)
    y_pred=model.predict(X_test)
    y_prob=model.predict_proba(X_test)
    print('F1:',f1_score(y_test,y_pred))
    print('Precision:',precision_score(y_test,y_pred))
    print('Recall:',recall_score(y_test,y_pred))
    print('Confusion matrix:\n',confusion_matrix(y_test,y_pred))
    print('Accuracy:',balanced_accuracy_score(y_test,y_pred))
    try:
        print('Roc AUC:',roc_auc_score(y_test,y_prob))
    except:
        print('ROC AUC: Nan')
    print('\n')

# SMOTE + UNDERSAMPLE 

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

models=[RandomForestClassifier(n_estimators=10,n_jobs=-1,bootstrap=True),
       GaussianNB(),LogisticRegression(),
        SGDClassifier('log',learning_rate='adaptive',eta0=0.001), DecisionTreeClassifier(),
        AdaBoostClassifier()]
model_names=['RF','GNB','LOG_Reg','SGDClass','DecTree','ADA']
for model,model_name in zip(models,model_names):
    print(model_name)
    print(Counter(y_train['Class']))
    oversample =SMOTENC(categorical_features=[1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,
                                            22,23,24,25,26,27,28,29,30,31],sampling_strategy=1,k_neighbors=1)
    undersample=RandomUnderSampler(0.5)
    
    X_train_s,y_train_s= undersample.fit_resample(X_train,y_train)
    X_train_s, y_train_s = oversample.fit_resample(X_train_s, y_train_s)
    
    print(Counter(y_train_s['Class']))
    model.fit(X_train_s,y_train_s)
    y_pred=model.predict(X_test)
    print('F1:',f1_score(y_test,y_pred))
    print('Precision:',precision_score(y_test,y_pred))
    print('Recall:',recall_score(y_test,y_pred))
    print('Confusion matrix:\n',confusion_matrix(y_test,y_pred))
    print('Accuracy:',balanced_accuracy_score(y_test,y_pred))
    try:
        print('Roc AUC:',roc_auc_score(y_test,y_pred))
    except:
        print('ROC AUC: Nan')
    print('\n')

# feature selection with K-Fold cross val on training set.
# test performances on test set

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import mutual_info_classif
models=[RandomForestClassifier(n_estimators=10,n_jobs=-1,bootstrap=True,class_weight='balanced_subsample'),
       GaussianNB(),LogisticRegression(class_weight='balanced'),DummyClassifier(strategy='most_frequent'),
        SGDClassifier('log',learning_rate='adaptive',eta0=0.01,class_weight='balanced'), DecisionTreeClassifier(class_weight='balanced'),
        AdaBoostClassifier(n_estimators=10),GradientBoostingClassifier(n_estimators=10)]

model_names=['RF','GNB','LOG_Reg','Dummy','SGDClass','DecTree','ADA','GBC']
n_feat=10
for model,model_name in zip(models,model_names):
    feat={key:0 for key in X_train.columns}
    print(model_name)
    skf=StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
    for train_index, test_index in skf.split(X_train, y_train):
        X_train_K, X_test_K = X_train.iloc[train_index,:], X_train.iloc[test_index,:]
        y_train_K, y_test_K = y_train.iloc[train_index,:], y_train.iloc[test_index,:]
        selector = SelectKBest(mutual_info_classif, k=n_feat)
        selector.fit(X_train_K, y_train_K)
        support = selector.get_support()
        for j in X_train_K.loc[:,support].columns.to_list():
            feat[j]+=1
    feat={k: v for k, v in sorted(feat.items(), key=lambda feat: feat[1],reverse=True)}
    support=list(feat.keys())[:n_feat+1]
    X_train_feat=X_train.loc[:,support]
    X_test_feat=X_test.loc[:,support]
    model.fit(X_train_feat,y_train)
    y_pred=model.predict(X_test_feat)
    y_prob=model.predict_proba(X_test_feat)
    print('F1:',f1_score(y_test,y_pred))
    print('Precision:',precision_score(y_test,y_pred))
    print('Recall:',recall_score(y_test,y_pred))
    print('Confusion matrix:\n',confusion_matrix(y_test,y_pred))
    print('Accuracy:',balanced_accuracy_score(y_test,y_pred))
    try:
        print('Roc AUC:',roc_auc_score(y_test,y_prob))
    except:
        print('ROC AUC: Nan')
    print('\n')
    
    
        

# K-Fold cross validation

In [ ]:
from sklearn.model_selection import StratifiedKFold,RepeatedKFold
from sklearn.feature_selection import mutual_info_classif

models=[RandomForestClassifier(n_estimators=10,n_jobs=-1,bootstrap=True,class_weight='balanced_subsample'),
       GaussianNB(),LogisticRegression(class_weight='balanced'),DummyClassifier(strategy='most_frequent'),
        SGDClassifier('log',learning_rate='adaptive',eta0=0.01,class_weight='balanced'), DecisionTreeClassifier(class_weight='balanced'),
        AdaBoostClassifier(n_estimators=10),GradientBoostingClassifier(n_estimators=10)]

model_names=['RF','GNB','LOG_Reg','Dummy','SGDClass','DecTree','ADA','GBC']
for model,model_name in zip(models,model_names):
    acc,f1,p,r,accb,roc = [],[],[],[],[],[]
    print(model_name)
    skf=StratifiedKFold(n_splits=7,random_state=0,shuffle=True)
    for train_index, test_index in skf.split(X, Y):
        X_train_K, X_test_K = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train_K, y_test_K = Y.iloc[train_index,:], Y.iloc[test_index,:]
        # select best features on training 
        #print(list(X_train_K.columns))
    
        model = model.fit(X_train_K,y_train_K.values.ravel())
        y_pred = model.predict(X_test_K)
        y_prob=model.predict_proba(X_test_K)[:,1]
        acc.append(accuracy_score(y_test_K,y_pred))
        f1.append(f1_score(y_test_K,y_pred))
        p.append(precision_score(y_test_K,y_pred))
        r.append(recall_score(y_test_K,y_pred))
        accb.append(balanced_accuracy_score(y_test_K,y_pred))
        try:
            roc.append(roc_auc_score(y_test_K,y_prob))
        except:
            roc.append(np.nan)
        

    print('Acc mean',np.mean(acc))    
    print('F1 mean',np.mean(f1))
    print('Precision mean',np.mean(p))
    print('Recall mean',np.mean(r))
    print('Balance Acc mean',np.mean(accb))
    try:
        print('Roc AUC:',np.mean(roc))
    except:
        print('ROC AUC: Nan')
    print('\n')

# K-Fold Cross validation + smote

In [ ]:

models=[RandomForestClassifier(n_estimators=10,n_jobs=-1,bootstrap=True,class_weight='balanced_subsample'),
       GaussianNB(),LogisticRegression(class_weight='balanced'),DummyClassifier(strategy='most_frequent'),
        SGDClassifier('log',learning_rate='adaptive',eta0=0.01,class_weight='balanced'), DecisionTreeClassifier(class_weight='balanced'),
        AdaBoostClassifier(n_estimators=10),GradientBoostingClassifier(n_estimators=10)]

model_names=['RF','GNB','LOG_Reg','Dummy','SGDClass','DecTree','ADA','GBC']
for model,model_name in zip(models,model_names):
    acc,f1,p,r,accb,roc = [],[],[],[],[],[]
    print(model_name)
    skf=StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
    for train_index, test_index in skf.split(X, Y):
        X_train_K, X_test_K = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train_K, y_test_K = Y.iloc[train_index,:], Y.iloc[test_index,:]
                


        oversample =SMOTENC(categorical_features=[1,2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,21,
                                            22,23,24,25,26,27,28,29,30],sampling_strategy=1,k_neighbors=1)
        undersample=RandomUnderSampler(0.5)
        X_train_K, y_train_K = undersample.fit_resample(X_train_K, y_train_K)
        X_train_K, y_train_K = oversample.fit_resample(X_train_K, y_train_K)
        model = model.fit(X_train_K,y_train_K.values.ravel())
        y_pred = model.predict(X_test_K)
        y_prob=model.predict_proba(X_test_K)[:,1]
        acc.append(accuracy_score(y_test_K,y_pred))
        f1.append(f1_score(y_test_K,y_pred))
        p.append(precision_score(y_test_K,y_pred))
        r.append(recall_score(y_test_K,y_pred))
        accb.append(balanced_accuracy_score(y_test_K,y_pred))
        try:
            roc.append(roc_auc_score(y_test_K,y_prob))
        except:
            roc.append(np.nan)


    print('Acc mean',np.mean(acc))    
    print('F1 mean',np.mean(f1))
    print('Precision mean',np.mean(p))
    print('Recall mean',np.mean(r))
    print('Balance Acc mean',np.mean(accb))
    try:
        print('Roc AUC:',np.mean(roc))
    except:
        print('ROC AUC: Nan')
    print('\n')

# K-Fold +SMOTE+ feat

In [ ]:
from sklearn.model_selection import StratifiedKFold,RepeatedKFold,LeaveOneOut
from sklearn.feature_selection import RFE
from sklearn.feature_selection import mutual_info_classif
from scipy.stats import mannwhitneyu
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE,SMOTENC
#model=DecisionTreeClassifier(class_weight='balanced')
#model=GradientBoostingClassifier(n_estimators=100)
#X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=1,shuffle=True)
model=LogisticRegression(class_weight='balanced')
#model=GaussianNB()
#model=RandomForestClassifier(n_estimators=100,n_jobs=-1,bootstrap=True,class_weight='balanced_subsample')
#model=SGDClassifier('log',learning_rate='adaptive',eta0=0.005,class_weight='balanced')
n_feat=[1,2,3,4,5,6,7,8,9,10]
#X=X.drop(columns=['Diff_BL_1yr'])
for f in n_feat:
    print('\nN Feat:',f)
    
    skf=StratifiedKFold(n_splits=10,random_state=1,shuffle=False)
    skf1=StratifiedKFold(n_splits=10,random_state=1,shuffle=False)
    loo=LeaveOneOut()
    acc,f1,p,r,accb,roc,brier,mcc = [],[],[],[],[],[],[],[]
    tot_feat={key:0 for key in X.columns}    
    for train_index, test_index in skf1.split(X, Y):
        X_train, X_test = X.iloc[train_index,:], X.iloc[test_index,:]
        y_train, y_test = Y.iloc[train_index,:], Y.iloc[test_index,:]
        y_p=[]
        y_t=[]
        feat={key:0 for key in X.columns}
        for train_index, test_index in skf.split(X_train, y_train):
            X_train_K, X_test_K = X_train.iloc[train_index,:], X_train.iloc[test_index,:]
            y_train_K, y_test_K = y_train.iloc[train_index,:], y_train.iloc[test_index,:]


            ###VARIABLE RANKING
            rank={key:0 for key in X_train.columns}
            for j in X_train.columns:
                rank[j]=mannwhitneyu(X_train_K[y_train_K['Class']==0][j],X_train_K[y_train_K['Class']==1][j])[1]
            rank={k: v for k, v in sorted(rank.items(), key=lambda item: item[1],reverse=False)}  
            support=list(rank.keys())[0:f]

            

            ###INCREMENT FEATURE COUNTER
            for j in X_train_K.loc[:,support].columns.to_list():
                feat[j]+=1
                tot_feat[j]+=1
                

            ###SELECT BEST N FEAT
            X_train_K = X_train_K.loc[:,support]


            ###SMOTE
            under=RandomUnderSampler(0.5,random_state=0)
            oversample=SMOTENC(sampling_strategy=1,categorical_features=[i for i in range(f)],k_neighbors=5,random_state=0)




        print('TEST')
        tot_feat={k: v for k, v in sorted(tot_feat.items(), key=lambda item: item[1],reverse=True)}
        feat={k: v for k, v in sorted(feat.items(), key=lambda item: item[1],reverse=True)}
        print(list(feat.keys())[0:f])

        X_train_ov=X_train.loc[:,list(feat.keys())[0:f]]
        X_test_ov=X_test.loc[:,list(feat.keys())[0:f]]
        
        X_train_ov,y_train_ov=under.fit_resample(X_train_ov, y_train)
        
        X_train_ov['temp']=0
        X_train_ov, y_train_ov = oversample.fit_resample(X_train_ov, y_train_ov)
        X_train_ov=X_train_ov.drop(columns=['temp'])
        
        model=model.fit(X_train_ov,y_train_ov)
        
        y_pred=model.predict(X_test_ov)
        y_prob=model.predict_proba(X_test_ov)[:,1]

        acc.append(accuracy_score(y_test,y_pred))
        f1.append(f1_score(y_test,y_pred))
        p.append(precision_score(y_test,y_pred))
        r.append(recall_score(y_test,y_pred))
        accb.append(balanced_accuracy_score(y_test,y_pred))
        roc.append(roc_auc_score(y_test,y_prob))
        brier.append(brier_score_loss(y_test,y_pred))
        mcc.append(matthews_corrcoef(y_test,y_pred))
        

    print('Acc mean',np.mean(acc))    
    print('F1 mean',np.mean(f1))
    print('Precision mean',np.mean(p))
    print('Recall mean',np.mean(r))
    print('Balance Acc mean',np.mean(accb))
    try:
        print('Roc AUC:',np.mean(roc))
    except:
        print('ROC AUC: Nan')
    print('Brier',np.mean(brier))
    print('MCC',np.mean(mcc))
    
    tot_feat={k: v for k, v in sorted(tot_feat.items(), key=lambda item: item[1],reverse=True)}
    print(tot_feat)

In [ ]:
print(X)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures as pol
from sklearn.pipeline import make_pipeline
model_pol=make_pipeline(pol(10),LinearRegression())
lab=prog.loc[:,['PATNO','Diff_BL_4yr']].dropna()
dat=whole.merge(lab,how='left',on='PATNO').dropna()
dat

In [ ]:
#dat=dat.drop(columns=['PATNO','Diff_BL_1yr','Class'])
X_reg=dat.iloc[:,:-1]
Y_reg=dat.iloc[:,-1]

In [ ]:
for col in X_reg.columns:
    if col=='gen' or col=='DOMSIDE':
        continue
    else:
        X_reg[col]=(X_reg[col]-X_reg[col].mean())/X_reg[col].std()


In [ ]:
FILE = "Desktop/ML_Chap.html"

with open(FILE, 'r') as html_file:
    content = html_file.read()

# Get rid off prompts and source code
content = content.replace("div.input_area {","div.input_area {\n\tdisplay: none;")    
content = content.replace(".prompt {",".prompt {\n\tdisplay: none;")

f = open(FILE, 'w')
f.write(content)
f.close()